In [0]:
! pip install spacy
! python -m spacy download en
! pip install torchtext==0.2.3
! pip install tweet-preprocessor
!pip install keras

In [16]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

In [4]:
from pathlib import Path
# from fastai.text import *
# import preprocessor as pre
# pre.set_options(pre.OPT.URL,pre.OPT.NUMBER,pre.OPT.MENTION,pre.OPT.HASHTAG)

In [9]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline,linear_model
from sklearn.metrics import confusion_matrix
# from fastai.plots import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import re
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [61]:
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras

Using TensorFlow backend.


In [12]:
from nltk import word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
PATH=Path("../")

In [85]:
df=pd.read_csv(PATH/'data/data.csv')
df.head(3)

,title,score,id,body,author,flair,url,comms_num,created,comment,authors,combined_features
0,"Lost my Job, Sick Mother and Paralysed Dad, In...",1042,g014wc,Hi....It's really tough time for everyone. I r...,sanand_satwik,AskIndia,https://www.reddit.com/r/india/comments/g014wc...,132,1.586742e+09,I'm a freelancer. Don't listen to the idiots ...,hashedram diabapp xataari Aashayrao sarcrasti...,"Lost my Job, Sick Mother and Paralysed Dad, In..."
1,Why does the government come with a begging bo...,647,fxofyu,"We have floods, terrorist attacks, famines due...",TWO-WHEELER-MAFIA,AskIndia,https://www.reddit.com/r/india/comments/fxofyu...,205,1.586448e+09,I don't understand why they don't use money f...,Kinky-Monk ak32009 fools_eye None DwncstSheep...,Why does the government come with a begging bo...
2,Mother's condition is going worse due to hepat...,756,g0zlly,"Hi folks, I really appreciate the warm respons...",sanand_satwik,AskIndia,https://www.reddit.com/r/india/comments/g0zlly...,92,1.586871e+09,If anyone knows who is influential on Twitter...,AlternativeDrop6 TheRobotsHaveCome lanky32 pl...,Mother's condition is going worse due to hepat...


In [86]:
df['title'] = df['title'].astype(str)+df['body'].astype(str)+df['comment'].astype(str) 

In [87]:
df=df[['flair','title']]
df["flair"].apply(lambda x:str(x))
# df.dropna()

0       AskIndia
1       AskIndia
2       AskIndia
3       AskIndia
4       AskIndia
          ...   
1795      Sports
1796      Sports
1797      Sports
1798      Sports
1799      Sports
Name: flair, Length: 1800, dtype: object

In [88]:
all_flairs=list(np.unique(df['flair']))
df['flair']=df['flair'].apply(lambda x :all_flairs.index(x))
all_flairs=list(np.unique(df['flair']))

In [89]:
df["title"].apply(lambda x:str(x))

0       Lost my Job, Sick Mother and Paralysed Dad, In...
1       Why does the government come with a begging bo...
2       Mother's condition is going worse due to hepat...
3       Men who are 30+ and have decided not to get ma...
4       r/India: If money is no bar, would you prefer ...
                              ...                        
1795    Indian Women's and Men's Hockey Teams Seal Ber...
1796    'It's humiliating for us': village disowns Dut...
1797    Feroz Shah Kotla Renamed Arun Jaitley Stadium,...
1798    Indian cricket fans are the most irritating ha...
1799    Copy India's ambition to be the best: Ian Chap...
Name: title, Length: 1800, dtype: object

In [90]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['title'],df['flair'],random_state=2020)

In [91]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)

In [92]:
from sklearn.metrics import classification_report

In [93]:
print('===========================   Naive Bayes =====================')


vectorizers=[(count_vect,'count_vectorizer'),(tfidf_vect,'tfidf_vectorizer_word'),(tfidf_vect_ngram,'tfidf_vectorizer_word_ngram'),(tfidf_vect_ngram_chars,'tfidf_vectorizer_ngram_chars')]

#Naive Bayes for all features:
for vectorizer in vectorizers:
  print(vectorizer[1])
  clf = Pipeline([
    ('vect',vectorizer[0]),
    ('clf', MultinomialNB()),
  ])
  clf = clf.fit(X_train, y_train)
  predictions=clf.predict_proba(X_test)
  test_preds=np.argmax(predictions,axis=1)

  auc = classification_report(y_test, test_preds)
  print (auc)


===========================   Naive Bayes =====================
count_vectorizer
              precision    recall  f1-score   support

           0       0.24      0.89      0.38        44
           1       0.62      0.23      0.33        44
           2       0.92      0.47      0.62        51
           3       0.17      0.15      0.16        52
           4       0.93      0.25      0.40        51
           5       0.38      0.53      0.44        49
           6       0.48      0.76      0.59        55
           7       0.56      0.10      0.17        51
           8       1.00      0.38      0.55        53

    accuracy                           0.42       450
   macro avg       0.59      0.42      0.41       450
weighted avg       0.60      0.42      0.41       450

tfidf_vectorizer_word
              precision    recall  f1-score   support

           0       0.20      0.89      0.33        44
           1       0.61      0.32      0.42        44
           2       0.91      

In [94]:
print('===========================   Logistic Regression =====================')


vectorizers=[(count_vect,'count_vectorizer'),(tfidf_vect,'tfidf_vectorizer_word'),(tfidf_vect_ngram,'tfidf_vectorizer_word_ngram'),(tfidf_vect_ngram_chars,'tfidf_vectorizer_ngram_chars')]

for vectorizer in vectorizers:
  print(vectorizer[1])
  clf = Pipeline([
    ('vect',vectorizer[0]),
    ('clf', linear_model.LogisticRegression(multi_class='auto',solver='lbfgs')),
  ])
  clf = clf.fit(X_train, y_train)
  predictions=clf.predict_proba(X_test)
  test_preds=np.argmax(predictions,axis=1)

  auc = classification_report(y_test, test_preds)
  print (auc)

===========================   Logistic Regression =====================
count_vectorizer


/Users/apple/Desktop/ml/midas_internship/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

           0       0.62      0.45      0.53        44
           1       0.45      0.43      0.44        44
           2       0.74      0.57      0.64        51
           3       0.41      0.42      0.42        52
           4       0.76      0.80      0.78        51
           5       0.44      0.45      0.44        49
           6       0.63      0.62      0.62        55
           7       0.49      0.53      0.51        51
           8       0.66      0.87      0.75        53

    accuracy                           0.58       450
   macro avg       0.58      0.57      0.57       450
weighted avg       0.58      0.58      0.57       450

tfidf_vectorizer_word
              precision    recall  f1-score   support

           0       0.55      0.59      0.57        44
           1       0.55      0.50      0.52        44
           2       0.79      0.67      0.72        51
           3       0.50      0.44      0.47        52
  

In [95]:
print('===========================   SVM =====================')


vectorizers=[(count_vect,'count_vectorizer'),(tfidf_vect,'tfidf_vectorizer_word'),(tfidf_vect_ngram,'tfidf_vectorizer_word_ngram'),(tfidf_vect_ngram_chars,'tfidf_vectorizer_ngram_chars')]

# SVM on count vectors: SVM Classifier Pipeline on word count vector
for vectorizer in vectorizers:
  print(vectorizer[1])
  clf = Pipeline([
    ('vect',vectorizer[0]),
    ('clf', SVC(gamma='scale',probability=True)),
  ])
  clf = clf.fit(X_train, y_train)
  predictions=clf.predict_proba(X_test)
  test_preds=np.argmax(predictions,axis=1)

  auc = classification_report(y_test, test_preds)
  print (auc)

===========================   SVM =====================
count_vectorizer
              precision    recall  f1-score   support

           0       0.38      0.68      0.49        44
           1       0.20      0.20      0.20        44
           2       0.35      0.24      0.28        51
           3       0.32      0.17      0.23        52
           4       0.50      0.61      0.55        51
           5       0.32      0.24      0.28        49
           6       0.47      0.65      0.55        55
           7       0.30      0.06      0.10        51
           8       0.44      0.66      0.53        53

    accuracy                           0.39       450
   macro avg       0.37      0.39      0.36       450
weighted avg       0.37      0.39      0.36       450

tfidf_vectorizer_word
              precision    recall  f1-score   support

           0       0.52      0.55      0.53        44
           1       0.50      0.43      0.46        44
           2       0.83      0.59    